In [10]:
import numpy as np
import pandas as pd
from numba import njit
import vectorbt as vbt
import datetime

# 데이터 다운로드
symbols = ['AAPL', 'MSFT', 'GOOGL']
start_date = datetime.datetime(2020, 1, 1)
end_date = datetime.datetime(2023, 1, 1)

data = vbt.YFData.download(symbols, start=start_date, end=end_date)

close = data.get('Close')
high = data.get('High')
low = data.get('Low')

sma20 = vbt.MA.run(window=20, close=close)
sma60 = vbt.MA.run(window=60, close=close)
atr14 = vbt.IndicatorFactory.from_talib('ATR').run(high, low, close, timeperiod=14).real

entries = sma20.ma_crossed_above(sma60)
exits = sma20.ma_crossed_below(sma60)


returns = close.pct_change()
corr = returns.corr() # 이 값을 시간에 대해서 돌아가야 된다. order_func_nb함수 안에서 c에 대해서 돌아가는 것.

In [15]:
# 사용자 정의 주문 함수 정의
@njit
def order_func_nb(c, direction, fees, atr14):
    # 현재 가격
    price = c.close[c.i, c.col]
    # 현재 포지션 가치
    position_value = c.exec_state['position'] * price
    # 포지션 사이즈 계산 (현재 포지션 가치의 1%) / (2 * ATR20)
    if position_value > 0:
        pos_size = (position_value * 0.01) / (2 * atr14[c.i])
    else:
        pos_size = (c.exec_state['cash'] * 0.01) / (2 * atr14[c.i])

    # 손절 지점 설정 (진입 시점에서 2 * ATR20)
    stop_loss = price - 2 * atr14[c.i]

    # 주문 생성
    return vbt.order_nb(
        price=price,
        size=pos_size,
        direction=direction[c.col],
        fees=fees,
        stop=stop_loss
    )
# 호출 순서 설정 함수 정의
@njit
def pre_segment_func_nb(c):
    return vbt.sort_call_seq_nb(c, size=c.close.shape[0])

# 초기 데이터 설정
direction = np.array([1] * close.shape[1])  # long only

#포트폴리오 생성
portfolio = vbt.Portfolio.from_order_func(
    close=close,
    entries=entries,
    exits=exits,
    order_func_nb=order_func_nb,
    size=np.array([1] * len(close)),
    direction = direction,
    order_args=(atr14.values),#, corr_matrix.values),
    init_cash=100_000,
    fees=0.001,
    slippage=0.001,
    pre_segment_func_nb=pre_segment_func_nb,
    call_seq='auto',
    size_type='amount',
    stop_loss=2 * atr14.values,
    cash_sharing=True # 현금 공유
)

# 포트폴리오 분석 및 결과 출력
print(portfolio.stats())
portfolio.plot().show()

ValueError: CallSeqType.Auto must be implemented manually. Use sort_call_seq_nb in pre_segment_func_nb.

In [17]:
import numpy as np
import pandas as pd
from datetime import datetime
import talib
from numba import njit

import vectorbt as vbt
from vectorbt.utils.colors import adjust_opacity
from vectorbt.utils.enum_ import map_enum_fields
from vectorbt.base.reshape_fns import broadcast, flex_select_auto_nb, to_2d_array
from vectorbt.portfolio.enums import SizeType, Direction, NoOrder, OrderStatus, OrderSide
from vectorbt.portfolio import nb

size = pd.Series([1, -1, 1, -1])  # per row
price = pd.DataFrame({'a': [1, 2, 3, 4], 'b': [4, 3, 2, 1]})  # per element
direction = ['longonly', 'shortonly']  # per column
fees = 0.01  # per frame


@njit
def order_func_nb(c, size, direction, fees):
    return nb.order_nb(
        price=c.close[c.i, c.col],
        size=size[c.i],
        direction=direction[c.col],
        fees=fees
)

direction_num = map_enum_fields(direction, Direction)
pf = vbt.Portfolio.from_order_func(
    price,
    order_func_nb,
    np.asarray(size), np.asarray(direction_num), fees
)
pf.orders.records_readable


,Order Id,Column,Timestamp,Size,Price,Fees,Side
0,0,a,0,1.0,1.0,0.01,Buy
1,1,a,1,1.0,2.0,0.02,Sell
2,2,a,2,1.0,3.0,0.03,Buy
3,3,a,3,1.0,4.0,0.04,Sell
4,4,b,0,1.0,4.0,0.04,Sell
5,5,b,1,1.0,3.0,0.03,Buy
6,6,b,2,1.0,2.0,0.02,Sell
7,7,b,3,1.0,1.0,0.01,Buy


In [18]:
# Fetch price history
symbols = ['BTC-USD', 'ETH-USD', 'XRP-USD', 'BNB-USD', 'BCH-USD', 'LTC-USD']
start = '2020-01-01 UTC'  # crypto is UTC
end = '2020-09-01 UTC'
# OHLCV by column
ohlcv = vbt.YFData.download(symbols, start=start, end=end).concat()

pf_baseline = vbt.Portfolio.from_orders(
    ohlcv['Close'], size, price=ohlcv['Open'],
    init_cash='autoalign', fees=0.001, slippage=0.001, freq='d')
pf_baseline.sharpe_ratio()


@njit
def order_func_nb(c, size, price, fees, slippage):
    return nb.order_nb(
        size=nb.get_elem_nb(c, size),
        price=nb.get_elem_nb(c, price),
        fees=nb.get_elem_nb(c, fees),
        slippage=nb.get_elem_nb(c, slippage),
    )

@njit
def post_segment_func_nb(c, returns_out):
    returns_out[c.i, c.group] = c.last_return[c.group]

returns_out = np.empty_like(ohlcv['Close'], dtype=np.float_)
pf = vbt.Portfolio.from_order_func(
    ohlcv['Close'],
    order_func_nb,
    np.asarray(size),
    np.asarray(ohlcv['Open']),
    np.asarray(0.001),
    np.asarray(0.001),
    post_segment_func_nb=post_segment_func_nb,
    post_segment_args=(returns_out,),
    init_cash=pf_baseline.init_cash
)

returns = pf.wrapper.wrap(returns_out)
del pf
returns.vbt.returns(freq='d').sharpe_ratio()

error: bad escape \d at position 7